In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Concatenate, Dot
from tensorflow.keras.models import Model
from baseline_utils import create_labels_and_train

2024-11-25 02:00:32.116785: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-25 02:00:32.119733: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-25 02:00:32.128510: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732500032.142832   96354 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732500032.147144   96354 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-25 02:00:32.163751: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [2]:
users_final = pd.read_csv('https://raw.githubusercontent.com/ardahk/amex/refs/heads/main/two-tower/users_final_numeric.csv')
products_final= pd.read_csv('https://raw.githubusercontent.com/ardahk/amex/refs/heads/main/two-tower/products_final_numeric.csv')

In [3]:
# drop 'product_id' from both dataframe
products_final = products_final.drop(columns=['name', 'sku', 'id', 'name_embedding', 'name_embedding_flat'])
users_final = users_final.apply(lambda x: x.astype(float))
products_final = products_final.apply(lambda x: x.astype(float))

In [4]:
print(f"{users_final.shape}")
print(f"{products_final.shape}")

(80000, 17)
(19696, 33)


### Model 1: Deep Dense 
User tower = Dense128 + Dense 64
Product tower = Dense128 + Dense 64

In [5]:
user_input = Input(shape=(users_final.shape[1],), name='user_input')
item_input = Input(shape=(products_final.shape[1],), name='item_input')
user_tower = Dense(128, activation='relu')(user_input)
user_tower = Dense(64, activation='relu')(user_tower)
item_tower = Dense(128, activation='relu')(item_input)
item_tower = Dense(64, activation='relu')(item_tower)
dot_product = Dot(axes=1)([user_tower, item_tower])
model1 = Model(inputs=[user_input, item_input], outputs=dot_product)
model1.compile(optimizer='adam', loss='binary_crossentropy')
#model1.summary()

batch_size = 500
num_epochs = 25

create_labels_and_train(users_final, products_final, model1, batch_size, num_epochs)

2024-11-25 02:00:34.552181: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
/home/codespace/.python/current/lib/python3.12/site-packages/keras/src/models/functional.py:225: UserWarning: The structure of `inputs` doesn't match the expected structure: ['user_input', 'item_input']. Received: the structure of inputs=('*', '*')
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 951ms/step - loss: 15.9424
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 15.9424
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 15.9424
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 15.9424
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 15.9424
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 15.9424
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 15.9424
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 15.9424
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 15.9424
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 15.9424
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 15.9424
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 15.9424
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 15.9424
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 15.9424
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 15.9424
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 15.9424
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 15.9424
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 15.9424
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s

### Model 2: Dense Medium
User tower = Dense128 

Product tower = Dense128

In [6]:
user_input = Input(shape=(users_final.shape[1],), name='user_input')
item_input = Input(shape=(products_final.shape[1],), name='item_input')
user_tower = Dense(128, activation='relu')(user_input)
item_tower = Dense(128, activation='relu')(item_input)
dot_product = Dot(axes=1)([user_tower, item_tower])
model2 = Model(inputs=[user_input, item_input], outputs=dot_product)
model2.compile(optimizer='adam', loss='binary_crossentropy')
#model2.summary()

create_labels_and_train(users_final, products_final, model2, batch_size, num_epochs)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 518ms/step - loss: 15.9424
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 15.9424
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 15.9424
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 15.9424
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 15.9424
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 15.9424
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - loss: 15.9424
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 15.9424
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 15.9424
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 15.9424
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - loss: 15.9424
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 15.9424
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 15.9424
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 15.9424
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 15.9105
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 15.9424
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 15.9424
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - loss: 15.9424
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s

### Model 3: Dense Deepest

User tower = Dense256 + Dense128 + Dense64 

Product tower = Dense256 + Dense128 + Dense64 

In [7]:
user_input = Input(shape=(users_final.shape[1],), name='user_input')
item_input = Input(shape=(products_final.shape[1],), name='item_input')

user_tower = Dense(256, activation='relu')(user_input)
user_tower = Dense(128, activation='relu')(user_tower)
user_tower = Dense(64, activation='relu')(user_tower)

item_tower = Dense(256, activation='relu')(item_input)
item_tower = Dense(128, activation='relu')(item_tower)
item_tower = Dense(64, activation='relu')(item_tower)

dot_product = Dot(axes=1)([user_tower, item_tower])

model3 = Model(inputs=[user_input, item_input], outputs=dot_product)
model3.compile(optimizer='adam', loss='binary_crossentropy')

create_labels_and_train(users_final, products_final, model3, batch_size, num_epochs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - loss: 15.9424
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 15.9424
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 15.9424
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 15.9424
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 15.9424
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 15.9424
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 15.9424
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 15.9424
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 15.9424
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 15.9105
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 15.9424
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 15.9424
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 15.9424
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 15.9424
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 15.9424
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 15.9424
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 15.9424
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 15.9424
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18

### Model 4: Dense Large

User tower = Dense256 + Dense128 + Dense64 

Product tower = Dense256 + Dense128 + Dense64 